In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/video-id-2-img/video_id2img_id.json
/kaggle/input/clip-feature/clip-features-32/L06_V007.npy
/kaggle/input/clip-feature/clip-features-32/L05_V014.npy
/kaggle/input/clip-feature/clip-features-32/L02_V019.npy
/kaggle/input/clip-feature/clip-features-32/L10_V027.npy
/kaggle/input/clip-feature/clip-features-32/L11_V009.npy
/kaggle/input/clip-feature/clip-features-32/L10_V013.npy
/kaggle/input/clip-feature/clip-features-32/L04_V007.npy
/kaggle/input/clip-feature/clip-features-32/L01_V016.npy
/kaggle/input/clip-feature/clip-features-32/L01_V031.npy
/kaggle/input/clip-feature/clip-features-32/L11_V027.npy
/kaggle/input/clip-feature/clip-features-32/L01_V019.npy
/kaggle/input/clip-feature/clip-features-32/L02_V006.npy
/kaggle/input/clip-feature/clip-features-32/L12_V002.npy
/kaggle/input/clip-feature/clip-features-32/L05_V016.npy
/kaggle/input/clip-feature/clip-features-32/L06_V018.npy
/kaggle/input/clip-feature/clip-features-32/L12_V027.npy
/kaggle/input/clip-feature/clip-featur

In [3]:
%pip install faiss-gpu
%pip install openai-clip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-0_57i8cx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-0_57i8cx
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install open_clip_torch


Note: you may need to restart the kernel to use updated packages.


In [33]:
import faiss
import open_clip
import numpy as np
import json
import torch

class Text_Search:
    def __init__(self, clipv2_bin: str, keyframe_file_json: str):
        self.clip_index = self.load_bin_file(clipv2_bin)
        self.id2img_fps = self.load_json_file(keyframe_file_json)
        # Load model
        self._device = "cuda" if torch.cuda.is_available() else "cpu"
        self.clipv2_model, _, _ = open_clip.create_model_and_transforms('ViT-L-14', device=self._device, pretrained='datacomp_xl_s13b_b90k')
        self.clipv2_tokenizer = open_clip.get_tokenizer('ViT-L-14')

    def load_bin_file(self, bin_file):
        return faiss.read_index(bin_file)

    def load_json_file(self, json_file):
        with open(json_file, 'r') as f:
            js = json.load(f)
        return js

    def find_video_info(self, index_images):
        for keys, values in self.id2img_fps.items():
            if int(keys)== index_images:
                return values
        return None

    def text_search(self, queries: str, k: int, index=None):
        """Implement text search in database"""
        text = self.clipv2_tokenizer([queries]).to(self._device)
        text_features = self.clipv2_model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        text_features = text_features.cpu().detach().numpy().astype(np.float32)

        if index is None:
            scores, index_image = self.clip_index.search(text_features, k=k)
        else:
            id_selector = faiss.IDSelectorArray(index)
            scores, index_image = self.clip_index.search(text_features, k=k, params=faiss.SearchParametersIVF(id_selector))

        idx_image = index_image.flatten()
        infos_query = [self.find_video_info(idx) for idx in idx_image]  # Ensure idx is a valid index
        link_paths= [info["image_path"] for info in infos_query]
        return scores.flatten(), idx_image,link_paths


In [29]:
import json

def load_json_file(json_file: str):
    with open(json_file, "r") as f:
        js = json.load(f)
    return js  # Return the loaded JSON data, not the file object

def find_video_info(json_file: str, index: int):
    video_data = load_json_file(json_file)
    for keys,value in video_data.items():
        if int(keys) ==index:
            return value 
    return None

# Example usage
json_file = r"/kaggle/input/id2img-fps/id2img_fps.json"
print(find_video_info(json_file, 128924))


{'image_path': '/mnt/f/Luan/AIHCM/KeyFrames/L04_extra/V012/011015.jpg', 'scene_idx': 'L04/V012/lst_shot/136'}


In [35]:
%pip install googletrans

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15718 sha256=233277881cbff30d10c52af2f616f5e7fef3f7ffe07a53f529cc71657ae3cc38
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Success

In [34]:
link_bin_file = r"/kaggle/input/bin-file/faiss_clipv2_cosine.bin"
json_file = r"/kaggle/input/id2img-fps/id2img_fps.json"

text_searcher = Text_Search(link_bin_file, json_file)
input_test = " "
scores, idx_image , image_path= text_searcher.text_search(input_test, k=10, index=None)  # Ensure all three values are returned

print("Image Indexes:", idx_image)

print("Image path ", image_path)

Image Indexes: [ 77722 192833  77686  77685 392138  77721  77720  69583  77889  77684]
Image path  ['/mnt/f/Luan/AIHCM/KeyFrames/L03_extra/V001/011763.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L06_extra/V001/013548.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L03_extra/V001/010938.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L03_extra/V001/010884.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L10_extra/V026/017244.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L03_extra/V001/011634.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L03_extra/V001/011505.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L02_extra/V026/010733.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L03_extra/V001/018119.jpg', '/mnt/f/Luan/AIHCM/KeyFrames/L03_extra/V001/010830.jpg']
